In [1]:
"""
Orbital Models of High Velocity Stars in Omega Centauri
Using Octofitterpy
"""

'\nOrbital Models of High Velocity Stars in Omega Centauri\nUsing Octofitterpy\n'

In [2]:
#Environment variables
%env JULIA_NUM_THREADS=auto
%env OCTOFITTERPY_AUTOLOAD_EXTENSIONS=yes

env: JULIA_NUM_THREADS=auto
env: OCTOFITTERPY_AUTOLOAD_EXTENSIONS=yes


In [3]:
#Imports 
import numpy as np
import astropy.units as u
from astropy.time import Time
import matplotlib.pylab as plt
import matplotlib.colors as colors
from PyAstronomy import pyasl
import astropy.constants as const
import pandas as pd
import matplotlib.lines as mlines
from astropy.table import Table
from datetime import datetime
from scipy import stats
import matplotlib.cm as cm
import matplotlib.colorbar as colorbar


import sys
sys.path
sys.path.append(r"C:\Users\macke\OneDrive - Saint Marys University\Summer Research 2025\two_body\orbit_utilities")
print(sys.path)
import two_body_utils as utils
import octo_utils 

import octofitterpy as octo

['C:\\Users\\macke\\anaconda3\\envs\\octo\\python311.zip', 'C:\\Users\\macke\\anaconda3\\envs\\octo\\Lib', 'C:\\Users\\macke\\anaconda3\\envs\\octo\\DLLs', 'C:\\Users\\macke\\anaconda3\\envs\\octo', '', 'C:\\Users\\macke\\anaconda3\\envs\\octo\\Lib\\site-packages', 'C:\\Users\\macke\\anaconda3\\envs\\octo\\Lib\\site-packages\\win32', 'C:\\Users\\macke\\anaconda3\\envs\\octo\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\macke\\anaconda3\\envs\\octo\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\macke\\OneDrive - Saint Marys University\\Summer Research 2025\\two_body\\orbit_utilities']
Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
Detected Jupyter notebook. Loading juliacall extension. Set `OCTOFITTERPY_AUTOLOAD_EXTENSIONS=no` to disable.


['C:\\Users\\macke\\anaconda3\\envs\\octo\\python311.zip', 'C:\\Users\\macke\\anaconda3\\envs\\octo\\Lib', 'C:\\Users\\macke\\anaconda3\\envs\\octo\\DLLs', 'C:\\Users\\macke\\anaconda3\\envs\\octo', '', 'C:\\Users\\macke\\anaconda3\\envs\\octo\\Lib\\site-packages', 'C:\\Users\\macke\\anaconda3\\envs\\octo\\Lib\\site-packages\\win32', 'C:\\Users\\macke\\anaconda3\\envs\\octo\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\macke\\anaconda3\\envs\\octo\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\macke\\OneDrive - Saint Marys University\\Summer Research 2025\\two_body\\orbit_utilities', 'C:\\Users\\macke\\OneDrive - Saint Marys University\\Summer Research 2025\\two_body\\orbit_utilities']


In [4]:
# === 1. Select stars and time config ===
star_names = ["A", "B","C","D","E","F","G"]
epoch = 2010.0
dt = 1

# Dictionaries to store simulation results and likelihood objects
epochs_mjd = {}
ra_rel = {}
dec_rel = {}
ra_errs = {}
dec_errs = {}

astrom_likelihoods = {}

# === 2. Simulate astrometry and create likelihood objects ===
for name in star_names:
    star = octo_utils.stars[name]
    emjd, ra_r, dec_r, ra_e, dec_e = octo_utils.simulate_astrometry(star=star, epoch=epoch, dt=dt)
    
    epochs_mjd[name] = emjd
    ra_rel[name] = ra_r
    dec_rel[name] = dec_r
    ra_errs[name] = ra_e
    dec_errs[name] = dec_e

    astrom_likelihoods[name] = octo.PlanetRelAstromLikelihood(
        epoch=epochs_mjd[name],             # Observation epochs (Modified Julian Dates)
        ra=ra_rel[name].tolist(),           # Relative Right Ascension values [mas]
        dec=dec_rel[name].tolist(),         # Relative Declination values [mas]
        σ_ra=ra_errs[name],                 # Measurement uncertainties in RA [mas]
        σ_dec=dec_errs[name],               # Measurement uncertainties in Dec [mas]
        cor=[0.0] * len(epochs_mjd[name])   # RA/Dec error correlation (zero for all epochs)
    )

# Access via astrom_likelihoods["A"]


In [5]:
dec_errs["A"]

[0.5030397697995657, 0.5, 0.5030397697995657]

In [6]:
"""
Companions 
"""
import octofitterpy as octo
planet_1 = octo.Planet(
    name = "A",                      # Name of the companion (used in output labels)
    basis = "Visual{KepOrbit}",              # Keplerian orbital basis for visual binary
    priors =
    """
        a ~ LogUniform(1, 100000)       # Semi-major axis [AU]
        e ~ Uniform(0.0, 0.99)         # Eccentricity
        i ~ Sine()                     # Inclination [rad]
        ω ~ UniformCircular()          # Argument of periastron [rad]
        Ω ~ UniformCircular()          # Longitude of ascending node [rad]
        θ ~ UniformCircular()          # Mean anomaly at reference epoch [rad]
        tp = θ_at_epoch_to_tperi(system, A, 55197.0)  # Reference MJD for θ (adjust to match your data)
    """,
    likelihoods=[octo.ObsPriorAstromONeil2019(astrom_likelihoods["A"]), astrom_likelihoods["A"]] 

)

planet_2 = octo.Planet(
    name = "B",                      # Name of the companion (used in output labels)
    basis = "Visual{KepOrbit}",              # Keplerian orbital basis for visual binary
    priors =
    """
        a ~ LogUniform(1, 100000)       # Semi-major axis [AU]
        e ~ Uniform(0.0, 0.99)         # Eccentricity
        i ~ Sine()                     # Inclination [rad]
        ω ~ UniformCircular()          # Argument of periastron [rad]
        Ω ~ UniformCircular()          # Longitude of ascending node [rad]
        θ ~ UniformCircular()          # Mean anomaly at reference epoch [rad]
        tp = θ_at_epoch_to_tperi(system, B, 55197.0)  # Reference MJD for θ (adjust to match your data)
    """,
    likelihoods=[octo.ObsPriorAstromONeil2019(astrom_likelihoods["B"]), astrom_likelihoods["B"]] 
)
planet_3 = octo.Planet(
    name = "C",                      # Name of the companion (used in output labels)
    basis = "Visual{KepOrbit}",              # Keplerian orbital basis for visual binary
    priors =
    """
        a ~ LogUniform(1, 100000)       # Semi-major axis [AU]
        e ~ Uniform(0.0, 0.99)         # Eccentricity
        i ~ Sine()                     # Inclination [rad]
        ω ~ UniformCircular()          # Argument of periastron [rad]
        Ω ~ UniformCircular()          # Longitude of ascending node [rad]
        θ ~ UniformCircular()          # Mean anomaly at reference epoch [rad]
        tp = θ_at_epoch_to_tperi(system, C, 55197.0)  # Reference MJD for θ (adjust to match your data)
    """,
    likelihoods=[octo.ObsPriorAstromONeil2019(astrom_likelihoods["C"]), astrom_likelihoods["C"]] 
)

planet_4 = octo.Planet(
    name = "D",                      # Name of the companion (used in output labels)
    basis = "Visual{KepOrbit}",              # Keplerian orbital basis for visual binary
    priors =
    """
        a ~ LogUniform(1, 100000)       # Semi-major axis [AU]
        e ~ Uniform(0.0, 0.99)         # Eccentricity
        i ~ Sine()                     # Inclination [rad]
        ω ~ UniformCircular()          # Argument of periastron [rad]
        Ω ~ UniformCircular()          # Longitude of ascending node [rad]
        θ ~ UniformCircular()          # Mean anomaly at reference epoch [rad]
        tp = θ_at_epoch_to_tperi(system, D, 55197.0)  # Reference MJD for θ (adjust to match your data)
    """,
    likelihoods=[octo.ObsPriorAstromONeil2019(astrom_likelihoods["D"]), astrom_likelihoods["D"]] 
)

planet_5 = octo.Planet(
    name = "E",                      # Name of the companion (used in output labels)
    basis = "Visual{KepOrbit}",              # Keplerian orbital basis for visual binary
    priors =
    """
        a ~ LogUniform(1, 100000)       # Semi-major axis [AU]
        e ~ Uniform(0.0, 0.99)         # Eccentricity
        i ~ Sine()                     # Inclination [rad]
        ω ~ UniformCircular()          # Argument of periastron [rad]
        Ω ~ UniformCircular()          # Longitude of ascending node [rad]
        θ ~ UniformCircular()          # Mean anomaly at reference epoch [rad]
        tp = θ_at_epoch_to_tperi(system, E, 55197.0)  # Reference MJD for θ (adjust to match your data)
    """,
    likelihoods=[octo.ObsPriorAstromONeil2019(astrom_likelihoods["E"]), astrom_likelihoods["E"]] 
)

planet_6 = octo.Planet(
    name = "F",                      # Name of the companion (used in output labels)
    basis = "Visual{KepOrbit}",              # Keplerian orbital basis for visual binary
    priors =
    """
        a ~ LogUniform(1, 100000)       # Semi-major axis [AU]
        e ~ Uniform(0.0, 0.99)         # Eccentricity
        i ~ Sine()                     # Inclination [rad]
        ω ~ UniformCircular()          # Argument of periastron [rad]
        Ω ~ UniformCircular()          # Longitude of ascending node [rad]
        θ ~ UniformCircular()          # Mean anomaly at reference epoch [rad]
        tp = θ_at_epoch_to_tperi(system, F, 55197.0)  # Reference MJD for θ (adjust to match your data)
    """,
    likelihoods=[octo.ObsPriorAstromONeil2019(astrom_likelihoods["F"]), astrom_likelihoods["F"]] 
)

planet_7 = octo.Planet(
    name = "G",                      # Name of the companion (used in output labels)
    basis = "Visual{KepOrbit}",              # Keplerian orbital basis for visual binary
    priors =
    """
        a ~ LogUniform(1, 100000)       # Semi-major axis [AU]
        e ~ Uniform(0.0, 0.99)         # Eccentricity
        i ~ Sine()                     # Inclination [rad]
        ω ~ UniformCircular()          # Argument of periastron [rad]
        Ω ~ UniformCircular()          # Longitude of ascending node [rad]
        θ ~ UniformCircular()          # Mean anomaly at reference epoch [rad]
        tp = θ_at_epoch_to_tperi(system, G, 55197.0)  # Reference MJD for θ (adjust to match your data)
    """,   
    likelihoods=[octo.ObsPriorAstromONeil2019(astrom_likelihoods["G"]), astrom_likelihoods["G"]] 
)

In [7]:
"""
Orbit Fitting and Plotting with Octofitter
"""


# === 3. Define the full system ===
# The system includes the host star’s mass and parallax, and any orbiting companions.

sys = octo.System(
    name = "Omega_Cen",             # System name (used for output files/plot titles)
    priors = 
    """
        M ~ Uniform(100, 200000)    # Host mass [solar masses]
       plx ~ truncated(Normal(0.18, 0.002), lower=0)     # Parallax [mas]
    """,
    likelihoods = [],                 # No system-level likelihoods in this case
    companions = [planet_1]             # List of orbiting bodies
) 


#planet_2, planet_3, planet_4, planet_5, planet_6, planet_7

# === 4. Compile the log-probability model ===
# This prepares the model for sampling. Optional: select a different backend or enable autodiff.
model = octo.LogDensityModel(sys)

# === 5. Fit the model ===
chain, pt = octo.octofit_pigeons(model, n_rounds=12, n_chains=50, n_chains_variational=50)
print(chain)

# Optional: save the summary table to a file
summary_str = repr(chain)

# === 6. Generate diagnostic plots ===
# This will create a set of corner and orbital plots and save them automatically
# as PNG images (default). Use fname="custom_name.pdf" to override.
octo.octoplot(model, chain, colorbar = True)


┌ Warning: This model has priors that cannot be sampled IID.
└ @ OctofitterPigeonsExt C:\Users\macke\.julia\packages\Octofitter\3mclS\ext\OctofitterPigeonsExt\OctofitterPigeonsExt.jl:99
[ Info: Sampler running with multiple threads     : true
[ Info: Likelihood evaluated with multiple threads: false
[ Info: Determining initial positions and metric using pathfinder
┌ Info: Found a sample of initial positions
└   initial_logpost_range = (-49.895000997842345, -38.43725334273556)


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
  scans     restarts      Λ        Λ_var      time(s)    allc(B)  log(Z₁/Z₀)   min(α)     mean(α)    min(αₑ)   mean(αₑ) 
────────── ────────── ────────── ────────── ────────── ────────── ────────── ────────── ────────── ────────── ──────────
        2          0       8.52       8.06       3.94   4.31e+07  -1.66e+07          0      0.833          1          1 
        4          0       10.4       11.4       7.69   5.03e+05  -3.03e+06          0       0.78          1          1 
        8          0       13.2       11.6       16.1   9.07e+05  -4.95e+05          0       0.75          1          1 
       16          0       15.7       14.4       31.4   1.64e+06  -8.34e+04          0      0.696          1          1 
       32          0       17.7       16.8       61.3   2.63e+06  -3.53e+04          0      0.652          1          1 
       64          0       18.5 

JuliaError: TaskFailedException

    nested task error: ReadOnlyMemoryError()
    Stacktrace:
      [1] unsafe_store!
        @ .\pointer.jl:146 [inlined]
      [2] setindex!
        @ C:\Users\macke\.julia\packages\UnsafeArrays\WkpzL\src\unsafe_array.jl:60 [inlined]
      [3] macro expansion
        @ C:\Users\macke\.julia\packages\Octofitter\3mclS\src\likelihoods\system.jl:84 [inlined]
      [4] macro expansion
        @ C:\Users\macke\.julia\packages\Bumper\erETf\src\internals.jl:76 [inlined]
      [5] macro expansion
        @ C:\Users\macke\.julia\packages\Octofitter\3mclS\src\likelihoods\system.jl:136 [inlined]
      [6] macro expansion
        @ C:\Users\macke\.julia\packages\RuntimeGeneratedFunctions\2SjTC\src\RuntimeGeneratedFunctions.jl:161 [inlined]
      [7] macro expansion
        @ .\none:0 [inlined]
      [8] generated_callfunc
        @ .\none:0 [inlined]
      [9] RuntimeGeneratedFunction
        @ C:\Users\macke\.julia\packages\RuntimeGeneratedFunctions\2SjTC\src\Runt

In [ ]:
# Orbit Plot
orbit_plot = octo.octoplot(model,chain, show_physical_orbit = True, colorbar = True)

# === companion names ===
companion_names = "All"

# === Build filename ===
plot_name = f"octo_orbit_{companion_names}"
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = rf"C:\Users\macke\OneDrive - Saint Marys University\Summer Research 2025\Plots\{plot_name}_{timestamp}.png"

# Save the image bytes to a file
with open(filename, "wb") as f:
    f.write(orbit_plot.data)

display(orbit_plot)

In [ ]:
# Generate the corner plot
corner_plot = octo.octocorner(model, chain, small=True)

# === companion names ===
companion_names = "All"

# === Build filename ===
plot_name = f"octo_corner_{companion_names}"
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = rf"C:\Users\macke\OneDrive - Saint Marys University\Summer Research 2025\Plots\{plot_name}_{timestamp}.png"

# Save the image bytes to a file
with open(filename, "wb") as f:
    f.write(corner_plot.data)

display(corner_plot)

In [ ]:
# """
# Saving your chain:
# Variables can be retrieved from the chains using the following syntax:
#     sma_planet_b = chain["b_a",:,:] 
# The first index is the name of the variable in the model (as a string or symbol).
# Planet-specific parameters are prefixed with the planet's name and an underscore (e.g., 'A_a' for semi-major axis of planet A).
# """

# # --- Extract and print best-fit orbital parameters for planet A (16th, 50th, 84th percentiles) ---

# # Semi-major axis in astronomical units
# print("Semi-major axis [AU]:", np.percentile(chain["A_a"], [16, 50, 84]))

# # Eccentricity (unitless, range: 0–1)
# print("Eccentricity:", np.percentile(chain["A_e"], [16, 50, 84]))

# # Orbital inclination in degrees (0° = face-on, 90° = edge-on)
# print("Inclination [deg]:", np.percentile(chain["A_i"], [16, 50, 84]))

# # Argument of periastron (angle from ascending node to periastron, in degrees)
# print("Argument of periastron [deg]:", np.percentile(chain["A_ω"], [16, 50, 84]))

# # Longitude of the ascending node (angle of orbit's orientation in space)
# print("Longitude of ascending node [deg]:", np.percentile(chain["A_Ω"], [16, 50, 84]))

# # Time of periastron passage in Modified Julian Date (MJD)
# print("Time of periastron passage [MJD]:", np.percentile(chain["A_tp"], [16, 50, 84]))

# # Convert MJD to calendar years (ISO format) using Octofitter's Julian date conversion
# tp_mjds = np.percentile(chain["A_tp"], [16, 50, 84])
# tp_years_octo = [octo.mjd2date(float(mjd)) for mjd in tp_mjds]
# print("Time of periastron passage [years]:", tp_years_octo)

# # Total system mass (usually primary + companion) in solar masses
# print("Total Mass [Solar Masses]:", np.percentile(chain["M"], [16, 50, 84]))


In [ ]:
# """
# Functions
# """

# def fake_pos(pos, pm, acc, dt):
#     """
#     Gives a fake observed position using an observed angular position, velocity, and acceleration

#     Parameters:
#     - pos : Initial position (in mas)
#     - pm : Proper motion (in mas/yr)
#     - acc : Acceleration (in mas/yr²)
#     - dt : Time offset(s) from the reference epoch in years

#     Returns:
#     - pos_final : calculated position (in mas)
#     """
#     return pos + pm * dt + 0.5 * acc * dt**2

# def propagate_error(sigma_pm, sigma_acc, dt):
#     """
#     Propagates uncertainty in position due to uncertainties in
#     proper motion and acceleration over time. No uncertainty in initial position is available.

#     Parameters:
#     - sigma_pm Uncertainty in proper motion (mas/yr)
#     - sigma_acc : Uncertainty in acceleration (mas/yr²)
#     - dt : Time from reference epoch (in years)

#     Returns:
#     - sigma_pos : Uncertainty in predicted position at time dt (in mas)
#     """
#     term_mu = (dt * sigma_pm) ** 2
#     term_acc = (0.5 * dt**2 * sigma_acc) ** 2
#     return np.sqrt(term_mu + term_acc)


In [ ]:
# """
# Mock dataset from single epoch observations for star A
# """

# # Time offset between epochs in years
# dt = 10              
# # Decimal-year epochs
# epochs_years = np.array([2010 - dt, 2010, 2010 + dt])   
# # Convert to MJD
# epochs_mjd = [octo.years2mjd(float(y)) for y in epochs_years]

# # Observational Data 
# # Star A
# ra_A = (201.6967263*u.deg).to(u.mas).value
# dec_A = (-47.4795835*u.deg).to(u.mas).value
# pm_ra_A = 3.563 # mas year^-1
# pm_dec_A = 2.564 # mas year^-1
# acc_ra_A = -0.0069 # mas year^-2
# acc_dec_A = 0.0085 # mas year^-2
# # Omega Centauri Center 
# # We find the center to be at (α,δ) = (13:26:47.24, −47:28:46.45).
# # from https://iopscience.iop.org/article/10.1088/0004-637X/710/2/1032
# # reference 6 in Haberle et al. (2024; Nature, Vol. 631) 
# # We assume the IMBH is located at the cluster center
# ra_cm_deg = 201.696834*u.deg
# dec_cm_deg = -47.479569*u.deg
# # Omega Centauri centre in mas
# ra_cm_mas  = ra_cm_deg.to(u.mas).value
# dec_cm_mas = dec_cm_deg.to(u.mas).value

# # Uncertainties 
# sigma_pm_ra_A = 0.038 # mas year^-1
# sigma_pm_dec_A = 0.055 # mas year^-1
# sigma_acc_ra_A = 0.0083 # mas year^-2
# sigma_acc_dec_A = 0.0098 # mas year^-2


# # Mock future and past observations
# future_ra_A  = fake_pos(ra_A,  pm_ra_A,  acc_ra_A,  dt)
# future_dec_A = fake_pos(dec_A, pm_dec_A, acc_dec_A, dt)
# past_ra_A    = fake_pos(ra_A,  pm_ra_A,  acc_ra_A, -dt)
# past_dec_A   = fake_pos(dec_A, pm_dec_A, acc_dec_A, -dt)

# future_ra_A_err  = propagate_error(sigma_pm_ra_A,  sigma_acc_ra_A,  dt)
# future_dec_A_err = propagate_error(sigma_pm_dec_A, sigma_acc_dec_A, dt)
# past_ra_A_err    = propagate_error(sigma_pm_ra_A,  sigma_acc_ra_A, -dt)
# past_dec_A_err   = propagate_error(sigma_pm_dec_A, sigma_acc_dec_A, -dt)

# # Arrays of absolute positions 
# ra_vals_A  = np.array([past_ra_A,  ra_A,  future_ra_A])   # mas
# dec_vals_A = np.array([past_dec_A, dec_A, future_dec_A])  # mas

# # Convert to positions relative to the assumed center of mass
# ra_rel_A  = ra_vals_A  - ra_cm_mas      # delta RA = companion − host  [mas]
# dec_rel_A = dec_vals_A - dec_cm_mas     # delta Dec = companion − host [mas]

# # Uncertainty is required for Octofitter for all positions (small uncertainty instead of zero)
# ra_errs_A = [past_ra_A_err, 1e-6, future_ra_A_err]
# dec_errs_A = [past_dec_A_err, 1e-6, future_dec_A_err]

# # Testing 
# print()
# print("Mock future RA of star A:", (future_ra_A*u.mas).to(u.deg), "±",(future_ra_A_err*u.mas).to(u.deg))
# print("Mock future DEC of star A:", (future_dec_A*u.mas).to(u.deg), "±",(future_dec_A_err*u.mas).to(u.deg))
# print()
# print("Mock past RA of star A:", (past_ra_A*u.mas).to(u.deg), "±",(past_ra_A_err*u.mas).to(u.deg))
# print("Mock past DEC of star A:", (past_dec_A*u.mas).to(u.deg), "±",(past_dec_A_err*u.mas).to(u.deg))
# print()

In [ ]:
# """
# Testing Values
# """
# # Manual values
# ra_vals_manual  = np.array([past_ra_A,  ra_A,  future_ra_A])
# dec_vals_manual = np.array([past_dec_A, dec_A, future_dec_A])
# ra_rel_manual   = ra_vals_manual - ra_cm_mas
# dec_rel_manual  = dec_vals_manual - dec_cm_mas
# ra_errs_manual  = [past_ra_A_err, 1e-6, future_ra_A_err]
# dec_errs_manual = [past_dec_A_err, 1e-6, future_dec_A_err]

# # octo_orbit values
# star = octo_utils.stars["A"]
# epoch = 2010.0
# dt = 10
# epochs_func, ra_rel_func, dec_rel_func, ra_errs_func, dec_errs_func = octo_utils.simulate_astrometry(star, epoch, dt)

# print("RA rel diff:", ra_rel_func - ra_rel_manual)
# print("Dec rel diff:", dec_rel_func - dec_rel_manual)
# print("RA err diff:", np.array(ra_errs_func) - np.array(ra_errs_manual))
# print("Dec err diff:", np.array(dec_errs_func) - np.array(dec_errs_manual))


In [ ]:
# """
# Orbital Calculations and Plots with Octofitter
# """
# # We start by defining a likelihood object for the relative astrometry of the star.
# astrom_like = octo.PlanetRelAstromLikelihood(
#     # MJD
#     epoch = epochs_mjd,      # Observation MJDs
#     # delta RA, milliarcseconds (East is positive)
#     ra = ra_rel_A.tolist(),      # Relative RA (mas)
#     # delta DEC, milliarcseconds (North is positive)
#     dec = dec_rel_A.tolist(),         # Relative Dec (mas)
#     # Uncertainty on RA, milliarcseconds
#     σ_ra = ra_errs_A,        # RA uncertainties (mas)
#     # Uncertainty on DEC, milliarcseconds
# 	σ_dec = dec_errs_A,      # Dec uncertainties (mas)
#     # Corelation between RA and DEC uncertainties
# 	cor= [0.0] * 3                    # No RA/Dec correlation
# )

# # We now define the orbiting star component of our model. 
# # We specify a name, an orbital basis, our priors, and list any likelihood objects.
# # Note! Change the reference epoch provided to θ_at_epoch_to_tperi from 50000 to one where your astrometry is defined
# star_A = octo.Planet(
#     name="A",
#     basis="Visual{KepOrbit}",
#     priors=
#     """            
#         a ~ LogUniform(0.1, 500)
#         e ~ Uniform(0.0, 0.99)
#         i ~ Sine()
#         ω ~ UniformCircular()
#         Ω ~ UniformCircular()
#         θ ~ UniformCircular()
#         tp = θ_at_epoch_to_tperi(system,A,55197.0) # use MJD epoch of your data here!!
#     """,
#     likelihoods=[astrom_like]
# )


# # Now we define our system, containing our star. 
# # We provide it with a name (used to specify the output file names and plot legends), 
# # our priors, a list of likelihood objects for the system as a whole (e.g., proper motion anomaly, radial velocity), 
# # and a list of companion models.
# # The name of your system determines the output file names
# sys = octo.System(
#     name="Omega_Cen_A",
#     priors = 
#     """
#         M ~ truncated(Normal(1.2, 0.1), lower=0)
#         plx ~ truncated(Normal(50.0, 0.02), lower=0)
#     """,
#     likelihoods=[],
#     companions=[star_A]
# )
# # We then compile our model. 
# # There are a few options for this function (for example, you can select the autodiff backend), but the defaults are reasonable.
# model = octo.LogDensityModel(sys)

# # We now fit the model. By default, this uses a Hamiltonian Monte Carlo sampler with 1000 steps of adaptation (discarded) and 1000 iterations. 
# # You can pass eg, adaptation=5000, iterations=25000, if you would like more points for your plots.
# # Long runtimes indiciate an error in your model
# # The orbit fitting should take <30 s for this example. 
# # If you find it takes much longer on your code, double check the model and priors. Is the epoch you specified for tperi the same as your data?
# # Numerical errors indicate a problem with sampling 
# # If you see a report that numerical errors were encountered during sampling, the results may not be statistically valid. 
# # You could re-try sampling, or investigate your model for errors.
# chain = octo.octofit(model)
# print(chain)

# # If you wish to save this summary table to a text file, you can convert it to a string like so:
# summary_str = repr(chain)

# # Use the octoplot function to generate plots. 
# # You can use keyword arguments (see documentation) to control which output panels appear. 
# # By default, they will be chosen automatically based on the kinds of data likelihoods you supply to the model.
# # The output is automatically saved to a PNG file in your current folder, with a name based on the name of the System you created. 
# # You can also customize this using the fname argument (see docs for more details). 
# # You can change the image format (eg to PDF) by changing the file name extension.
# octo.octoplot(model,chain)